In [77]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

from sklearn.metrics import r2_score

In [2]:
DF = pd.read_csv('clean_flats_df.csv')

In [8]:
DF.count()[:40]

Cсылка                  781628
Цена                    781639
Время загрузки          781639
Ближайшее метро         781639
Адрес                   780459
Этаж                    781639
Площадь кухни           781639
Расстояние до метро     781639
Количество комнат       781639
Жилая площадь           781639
Вид объекта             781639
Площадь                 781639
Город                   781639
Регион                  781639
ID                      781639
Тип объявления          781639
Этажей в доме           781633
lat                     781637
lng                     781637
Тип дома                781639
Год постройки           781639
Пассажирский лифт       781639
Грузовой лифт            41650
Запланирован снос       781639
Санузел                 781639
Ремонт                  781639
Статус                  781639
Высота потолков         781639
Вид из окон             781639
Способ продажи           99716
Балкон или лоджия       781639
Вид сделки               58035
Тёплый п

In [71]:
def prepare_for_lr(df):
    X = df[['Этаж', 'Площадь кухни', 'Расстояние до метро', 
       'Количество комнат', 'Жилая площадь', 'Площадь', 'Этажей в доме',
       'Тип дома', 'Год постройки', 'Запланирован снос', 'Санузел', 'Ремонт', 'Статус', 'Высота потолков',
       'Балкон или лоджия', 'Тёплый пол', 'Консьерж', 'Планировка', 'Ветка',
       'Округ', 'Плотность населения', 'Загруженность шоссе', 'Пассажирский лифт',
       'Площадь зеленых насаждений', 'Влияние объектов негативного воздействия']]
    X['Высота этажа'] = X['Этаж']/X['Этажей в доме']
    X['Высота этажа'] = X['Высота этажа'].map(lambda x: x if x <= 1 and x >= 0 else np.nan)
    X['Пассажирский лифт'] = X['Пассажирский лифт'].replace('Нет', 0).astype(int)
    y = np.log(df['Цена'])
    
    ordinary_cols = []
    binary_cols = []
    numeric_cols = []
    for col in X.columns:
        if type(X[col].loc[X[col].first_valid_index()]) == str:
            ordinary_cols.append(col)
        elif X[col].nunique() == 2:
            binary_cols.append(col)
        else:    
            numeric_cols.append(col)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                            random_state=42, stratify=X['Количество комнат'])
    
    for col in numeric_cols:
        X_train[col] = X_train[col].replace([np.inf, -np.inf], np.nan).fillna(X_train[col].mean())
        X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan).fillna(X_test[col].mean())
    
    for col in ordinary_cols:
        X_train[col] = X_train[col].fillna(X_train[col].mode())
        X_test[col] = X_test[col].fillna(X_test[col].mode()) 

    ohe = OneHotEncoder(drop = 'first',sparse = False, handle_unknown = 'ignore') 
    ohe.fit(X_train[ordinary_cols])
    ss = StandardScaler()
    ss.fit(X_train[numeric_cols])
    X_train = pd.concat([pd.DataFrame(ss.transform(X_train[numeric_cols]), columns = numeric_cols), 
        pd.DataFrame(ohe.transform(X_train[ordinary_cols]), columns = ohe.get_feature_names_out(ordinary_cols)),
                         X_train[binary_cols].reset_index(drop = True)], axis = 1)

    X_test = pd.concat([pd.DataFrame(ss.transform(X_test[numeric_cols]), columns = numeric_cols), 
        pd.DataFrame(ohe.transform(X_test[ordinary_cols]), columns = ohe.get_feature_names_out(ordinary_cols)),
                         X_test[binary_cols].reset_index(drop = True)], axis = 1)
    
    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = prepare_for_lr(DF)

In [80]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
r2_score(linreg.predict(X_train), y_train)

0.8009691183590709

In [81]:
r2_score(linreg.predict(X_test), y_test)

0.8011654650739289

In [95]:
import plotly.express as px
coef = pd.Series(linreg.coef_, index = X_train.columns)
px.histogram(coef, x = coef.index, y = coef.values, height = 750, width = 1500)